In [2]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import colors
import numpy as np
import statistical_operations as stat_ops

'''stop_img = basic_ops.load_image(image_path='traffic_sign_templates/stop_signs/stop.jpg')
vf_img = basic_ops.load_image(image_path='traffic_sign_templates/vf_signs/vf.jpg')
vfa_img = basic_ops.load_image(image_path='traffic_sign_templates/vfa_signs/vfa.jpg')
vfs_img = basic_ops.load_image(image_path='traffic_sign_templates/vfs_signs/vfs.jpg')'''
#images = basic_ops.load_images(folder_path='data', amount=3)
images = [basic_ops.load_image(image_path='traffic_sign_images/vfs_01.jpg')]
color_functions = [colors.is_strong_red, colors.is_strong_yellow, colors.is_strong_blue]
images = [geo_ops.resize_image(image=image, target_height=image.shape[0]//8, target_width=image.shape[1]//8) for image in images]

Image loaded from traffic_sign_images/vfs_01.jpg


In [3]:
for i, image in enumerate(images):
    image = filters.median_filter(image=image, dim=3)
    image = stat_ops.gamma_equalization(image=image, gamma=0.25)
    images[i] = image

In [4]:
color_bounding_boxes = []
color_images = images.copy()
for i, image in enumerate(color_images):
    image_bounding_boxes = []
    img = np.copy(image)
    for color_function in color_functions:
        mask = colors.get_mask(image=img, color_function=color_function)
        blobs = tsd.get_blobs(mask=mask)
        img, bounding_boxes = tsd.draw_bounding_boxes(image=img, blobs=blobs, min_box_area=200, box_color=colors.get_color_from_function(color_function))
        image_bounding_boxes.extend(bounding_boxes)
    color_bounding_boxes.extend(image_bounding_boxes)
    color_images[i] = img

[(283, 126), (283, 127), (284, 127), (284, 126), (285, 126), (285, 127), (285, 128), (286, 128), (287, 128), (287, 129), (288, 129), (288, 128)]
[(289, 336), (289, 337), (290, 337), (290, 336), (290, 338)]
[(293, 120), (294, 120), (294, 119), (295, 120)]
[(293, 132), (294, 132), (294, 131), (295, 131), (295, 130), (295, 129), (296, 129), (296, 128), (296, 127), (296, 126), (296, 125), (297, 128), (297, 129), (298, 129), (298, 128), (298, 130), (299, 130), (299, 129), (300, 130), (301, 130), (302, 130), (303, 130), (303, 129), (304, 129), (304, 128), (305, 128), (305, 127), (306, 127), (306, 128), (305, 129), (305, 130), (304, 130), (296, 130)]
[(307, 335), (307, 336)]
[(331, 60)]
[(332, 57), (332, 58), (332, 59), (333, 59), (333, 58), (333, 57), (333, 56), (333, 55)]
[(332, 67), (332, 68)]
[(334, 52), (334, 53), (335, 52), (335, 51), (335, 50), (335, 49), (336, 49), (336, 48), (336, 50), (336, 51), (337, 51), (337, 50), (337, 49)]
[(335, 54), (335, 55)]
[(336, 58), (337, 58)]
[(348, 11

In [5]:
for i, image in enumerate(color_images):
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip((image - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
    basic_ops.show_image(image=image)
    color_images[i] = image

Image displayed


In [6]:
edge_images = images.copy()
for i, image in enumerate(edge_images):
    image = filters.median_filter(image=image, dim=3)
    image = filters.gray_scale_filter(image=image)
    image = filters.sobel_filter(image=image, mode='both', threshold=90)
    edge_images[i] = image

In [7]:
from collections import deque

def find_contours(img, min_points):
    visited = np.zeros_like(img, dtype=bool)
    contours = []

    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if img[y, x] == 255 and not visited[y, x]:
                # BFS to find connected contour
                contour = []
                queue = deque()
                queue.append((y, x))
                visited[y, x] = True

                while queue:
                    cy, cx = queue.popleft()
                    contour.append((cx, cy))  # (x, y)

                    for dy in [-1, 0, 1]:
                        for dx in [-1, 0, 1]:
                            ny, nx = cy + dy, cx + dx
                            if (0 <= ny < img.shape[0]) and (0 <= nx < img.shape[1]):
                                if img[ny, nx] == 255 and not visited[ny, nx]:
                                    queue.append((ny, nx))
                                    visited[ny, nx] = True

                if len(contour) > min_points:  # filter small noise
                    contours.append(contour)
    return contours

def point_line_distance(point, start, end):
    if start == end:
        return np.linalg.norm(np.array(point) - np.array(start))
    return np.abs(np.cross(np.subtract(end, start), np.subtract(start, point))) / np.linalg.norm(np.subtract(end, start))

def rdp(points, epsilon):
    if len(points) < 3:
        return points

    start = points[0]
    end = points[-1]
    max_dist = 0
    index = 0
    for i in range(1, len(points) - 1):
        dist = point_line_distance(points[i], start, end)
        if dist > max_dist:
            max_dist = dist
            index = i

    if max_dist > epsilon:
        left = rdp(points[:index + 1], epsilon)
        right = rdp(points[index:], epsilon)
        return left[:-1] + right
    else:
        return [start, end]

def is_closed_contour(contour):
    return contour[0] == contour[-1]


def contour_area(contour):
    x_coords, y_coords = zip(*contour)
    return (max(x_coords) - min(x_coords)) * (max(y_coords) - min(y_coords))



In [13]:
import cv2
import traffic_sign_detection as tsd

for image in edge_images:
    basic_ops.show_image(image=image)
    contours = find_contours(image, 50)
    image = images[0]
    for contour in contours:
        if len(contour) > 5000:
            continue
        simplified = rdp(contour, epsilon=2.0)
        corners = len(simplified)

        if corners > 2:
            print(f'points: {simplified}')
            points_xy = [(x, y) for y, x in simplified]

            x_values = [p[0] for p in points_xy]
            y_values = [p[1] for p in points_xy]

            min_x, max_x = min(x_values), max(x_values)
            min_y, max_y = min(y_values), max(y_values)

            top_left = (min_x, min_y)
            bottom_right = (max_x, max_y)
            center_x = sum(x_values) // len(x_values)
            center_y = sum(y_values) // len(y_values)
            print(f'center: {center_y}, {center_x}')
            print(f'top_left: {top_left}, bottom_right: {bottom_right}')
            image = cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)

        if corners == 3:
            print("Triangle detected")
        elif corners == 4:
            print("Quadrilateral (could be diamond or square)")

        elif 7 <= corners <= 9:
            print("Circle-like detecteda")
    basic_ops.show_image(image=image)


Image displayed


/tmp/ipykernel_12245/1212868152.py:35: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  return np.abs(np.cross(np.subtract(end, start), np.subtract(start, point))) / np.linalg.norm(np.subtract(end, start))


points: [(35, 143), (33, 145), (37, 145), (32, 146), (38, 145), (31, 146), (39, 145), (30, 146), (40, 146), (29, 146), (41, 146), (28, 146), (42, 147), (27, 146), (43, 148), (26, 147), (44, 148), (26, 148), (45, 148), (24, 148), (46, 148), (23, 147), (27, 150), (47, 146), (22, 147), (48, 147), (21, 150), (49, 148), (20, 151), (50, 148), (20, 152), (51, 148), (20, 153), (52, 148), (19, 154), (53, 149), (18, 155), (54, 150), (17, 152), (19, 156), (54, 151), (16, 152), (53, 152), (15, 154), (53, 153), (14, 155), (52, 154), (13, 156), (52, 155), (12, 157), (52, 156), (12, 158), (52, 157), (12, 159), (58, 158)]
center: 34, 150
top_left: (143, 12), bottom_right: (159, 58)
points: [(68, 153), (66, 158), (70, 164), (67, 165), (70, 165), (67, 168), (76, 170), (77, 175), (80, 175), (80, 183)]
center: 72, 167
top_left: (153, 66), bottom_right: (183, 80)
points: [(32, 170), (29, 171), (32, 173), (28, 171), (32, 176), (34, 174), (33, 177), (35, 174), (34, 178), (36, 173), (35, 179), (37, 174), (35,